In [1]:
import pandas as pd
import os

In [2]:
SPLIT_CLASSES = {
    "PerceptionReasoning" : ["LRE","NRO","NDM","UTR"],
    "UrbanFlowComplexity" : [ "EMP","VRU","VCI","VRE"],
    "SafetyCompliance" : ["ORE","LSP","IRC","TPS","TSR","RSR"]
}
CLASS_LABELS = {
    1 : "SafetyCompliance" ,
    2 : "UrbanFlowComplexity",
    3 : "PerceptionReasoning"
}

In [79]:
PATH = "../Dataset/annotation.csv"
base_df = pd.read_csv(PATH)

base_df.head()

,index,id,image,question,A,B,C,D,answer,explanation
0,0,EMP01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"In this complex traffic scenario, what immedia...",The clear and open lane to the far left as a p...,The need to maintain a high speed to keep up w...,The potential for unpredictable movements from...,The stationary vehicles parked on the side of ...,C,The potential for unpredictable movements from...
1,1,EMP02,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"Considering the overall traffic environment, w...",The high speed of the surrounding traffic.,Difficulty in identifying the destination of t...,Limited available space due to numerous surrou...,Poor visibility due to the glare from streetli...,C,Limited available space due to numerous surrou...
2,2,EMP03,iVBORw0KGgoAAAANSUhEUgAAB38AAAP2CAYAAADn0VcdAA...,"As an autonomous driving agent, what is the mo...",Wait for the obstacle to clear while turning o...,Honk to signal the obstacle to move.,Attempt to drive around the obstacle on the ri...,Reverse and find an alternative route.,A,Wait for the obstacle to clear while turning o...
3,3,EMP04,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"Analyzing the depicted environment, what is th...","A sudden brake from the car in front, indicate...",There's nothing to look out for.,The car in front is preparing to accelerate qu...,A pedestrian suddenly crossing the road from t...,A,"A sudden brake from the car in front, indicate..."
4,4,EMP05,iVBORw0KGgoAAAANSUhEUgAABrIAAANvCAYAAACRd0pHAA...,"As an autonomous driving agent, what is the mo...",Pull over to ther left and wait until the vehi...,Quickly swerve to the right to bypass the vehi...,Honk the horn and proceed forward to assert ri...,Stop abruptly and wait for the vehicle behind ...,A,Pull over to ther left and wait until the vehi...


In [80]:
def split_df(df, classes):
    mask = df['id'].apply(lambda x: any(cls in x for cls in SPLIT_CLASSES[classes]))
    result_df = df[mask]
    return result_df

In [81]:
def reformat_df(df):
    # Initialize an empty list to store all rows (original + rotated copies)
    all_rows = []

    # Dictionary to map answer letters for each rotation
    answer_mapping = {
        0: {'A': 'A', 'B': 'B', 'C': 'C', 'D': 'D'},  # Original (no rotation)
        1: {'A': 'B', 'B': 'C', 'C': 'D', 'D': 'A'},  # Rotation 1: A→B, B→C, C→D, D→A
        2: {'A': 'C', 'B': 'D', 'C': 'A', 'D': 'B'},  # Rotation 2: A→C, B→D, C→A, D→B
        3: {'A': 'D', 'B': 'A', 'C': 'B', 'D': 'C'}   # Rotation 3: A→D, B→A, C→B, D→C
    }

    # Process each row in result_df
    for group_idx, row in enumerate(df.itertuples(), 1):
        # Create 4 versions: original + 3 rotations
        for rotation in range(4):
            new_row = row._asdict().copy()  # Convert row to dict
            new_row['g_index'] = group_idx  # Assign clustered group index
            
            # Rotate options based on rotation number
            if rotation == 0:  # Original
                new_row['A'], new_row['B'], new_row['C'], new_row['D'] = row.A, row.B, row.C, row.D
            elif rotation == 1:  # A→B, B→C, C→D, D→A
                new_row['A'], new_row['B'], new_row['C'], new_row['D'] = row.D, row.A, row.B, row.C
            elif rotation == 2:  # A→C, B→D, C→A, D→B
                new_row['A'], new_row['B'], new_row['C'], new_row['D'] = row.C, row.D, row.A, row.B
            elif rotation == 3:  # A→D, B→A, C→B, D→C
                new_row['A'], new_row['B'], new_row['C'], new_row['D'] = row.B, row.C, row.D, row.A
            
            # Update answer based on rotation
            new_row['answer'] = answer_mapping[rotation][row.answer]
            
            # Remove the original index (will be replaced with new sequential index)
            new_row.pop('index')
            
            # Append the new row to the list
            all_rows.append(new_row)

    # Create new DataFrame from all rows
    rotated_df = pd.DataFrame(all_rows)

    # Add new sequential index starting from 1
    rotated_df['index'] = range(1, len(rotated_df) + 1)

    # Reorder columns to place index and g_index first
    column_order = ['index', 'g_index', 'id', 'image', 'question', 'A', 'B', 'C', 'D', 'answer', 'explanation']
    rotated_df = rotated_df[column_order]
    
    return rotated_df

In [85]:
PATH = '../Dataset'
for i in range(1,4):
    labels = CLASS_LABELS[i]
    path = os.path.join(PATH, labels, f'{labels}_annotation.tsv')

    df = split_df(base_df, labels)
    df = reformat_df(df)
    df.to_csv(path, sep='\t', index=False)
    
    print(f'Successfully saved data in {path}')

Successfully saved data in ../Dataset/SafetyCompliance/SafetyCompliance_annotation.tsv
Successfully saved data in ../Dataset/UrbanFlowComplexity/UrbanFlowComplexity_annotation.tsv
Successfully saved data in ../Dataset/PerceptionReasoning/PerceptionReasoning_annotation.tsv


In [4]:
PATH = '../Dataset'
DATASET_PATH = []
for i in range(1,4):
    labels = CLASS_LABELS[i]
    path = os.path.join(PATH, labels, f'{labels}_annotation.tsv')
    DATASET_PATH.append(path)

perception_df = pd.read_csv(DATASET_PATH[0], sep='\t')
urban_df = pd.read_csv(DATASET_PATH[1], sep='\t')
safety_df = pd.read_csv(DATASET_PATH[2], sep='\t')


In [7]:
sample_df = perception_df.iloc[:16, :]
sample_df.to_csv('jktdrive.tsv', sep='\t', index=False)

In [90]:
perception_df

,index,g_index,id,image,question,A,B,C,D,answer,explanation
0,1,1,IRC01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"In this traffic scene, what action should the ...",Accelerate to avoid disrupting traffic flow.,"Slow down gradually and prepare to brake, main...",Turn on hazard lights and pull over immediately.,Follow the vehicle in front closely to use its...,B,"Slow down gradually and prepare to brake, main..."
1,2,1,IRC01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"In this traffic scene, what action should the ...",Follow the vehicle in front closely to use its...,Accelerate to avoid disrupting traffic flow.,"Slow down gradually and prepare to brake, main...",Turn on hazard lights and pull over immediately.,C,"Slow down gradually and prepare to brake, main..."
2,3,1,IRC01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"In this traffic scene, what action should the ...",Turn on hazard lights and pull over immediately.,Follow the vehicle in front closely to use its...,Accelerate to avoid disrupting traffic flow.,"Slow down gradually and prepare to brake, main...",D,"Slow down gradually and prepare to brake, main..."
3,4,1,IRC01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"In this traffic scene, what action should the ...","Slow down gradually and prepare to brake, main...",Turn on hazard lights and pull over immediately.,Follow the vehicle in front closely to use its...,Accelerate to avoid disrupting traffic flow.,A,"Slow down gradually and prepare to brake, main..."
4,5,2,IRC02,iVBORw0KGgoAAAANSUhEUgAAClAAAAXQCAYAAAAgNpFWAA...,Given the current road and environmental state...,Decelerate and wait until all other road users...,Swerve to the right and move forward.,Accelerate and drive through the streets.,Follow another vehicle's action closely.,A,Decelerate and wait until all other road users...
...,...,...,...,...,...,...,...,...,...,...,...
499,500,125,TSR12,iVBORw0KGgoAAAANSUhEUgAAB38AAAP9CAYAAACNFqfeAA...,"With the traffic scene ahead, what should the ...","Turn left cautiously, as the red light might n...",Remain stopped and wait for the main traffic l...,"Proceed straight, as the red light is only for...",Inch forward into the intersection to prepare ...,B,Remain stopped and wait for the main traffic l...
500,501,126,TSR13,iVBORw0KGgoAAAANSUhEUgAAB38AAAP7CAYAAABbT0TDAA...,"With the traffic scene ahead, what should the ...",Speed up to ensure you make it through the int...,Always slow down at a green light.,Continue moving forward cautiously while the l...,Stop to ensure the light is not about to change.,C,Continue moving forward cautiously while the l...
501,502,126,TSR13,iVBORw0KGgoAAAANSUhEUgAAB38AAAP7CAYAAABbT0TDAA...,"With the traffic scene ahead, what should the ...",Stop to ensure the light is not about to change.,Speed up to ensure you make it through the int...,Always slow down at a green light.,Continue moving forward cautiously while the l...,D,Continue moving forward cautiously while the l...
502,503,126,TSR13,iVBORw0KGgoAAAANSUhEUgAAB38AAAP7CAYAAABbT0TDAA...,"With the traffic scene ahead, what should the ...",Continue moving forward cautiously while the l...,Stop to ensure the light is not about to change.,Speed up to ensure you make it through the int...,Always slow down at a green light.,A,Continue moving forward cautiously while the l...


In [91]:
urban_df

,index,g_index,id,image,question,A,B,C,D,answer,explanation
0,1,1,EMP01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"In this complex traffic scenario, what immedia...",The clear and open lane to the far left as a p...,The need to maintain a high speed to keep up w...,The potential for unpredictable movements from...,The stationary vehicles parked on the side of ...,C,The potential for unpredictable movements from...
1,2,1,EMP01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"In this complex traffic scenario, what immedia...",The stationary vehicles parked on the side of ...,The clear and open lane to the far left as a p...,The need to maintain a high speed to keep up w...,The potential for unpredictable movements from...,D,The potential for unpredictable movements from...
2,3,1,EMP01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"In this complex traffic scenario, what immedia...",The potential for unpredictable movements from...,The stationary vehicles parked on the side of ...,The clear and open lane to the far left as a p...,The need to maintain a high speed to keep up w...,A,The potential for unpredictable movements from...
3,4,1,EMP01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"In this complex traffic scenario, what immedia...",The need to maintain a high speed to keep up w...,The potential for unpredictable movements from...,The stationary vehicles parked on the side of ...,The clear and open lane to the far left as a p...,B,The potential for unpredictable movements from...
4,5,2,EMP02,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"Considering the overall traffic environment, w...",The high speed of the surrounding traffic.,Difficulty in identifying the destination of t...,Limited available space due to numerous surrou...,Poor visibility due to the glare from streetli...,C,Limited available space due to numerous surrou...
...,...,...,...,...,...,...,...,...,...,...,...
323,324,81,VRU21,iVBORw0KGgoAAAANSUhEUgAAB38AAAPyCAYAAAB8QBULAA...,The agent is currently moving at a slowly stea...,Keep moving forward at a reduced speed with re...,Come to a complete stop and wait for all pedes...,Honk to make the pedestrians move faster.,Accelerate to avoid being delayed by pedestrians.,A,Keep moving forward at a reduced speed with re...
324,325,82,VRU22,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"The agent is intending to go to the left, but ...",They will leave the road as soon as the agent ...,They will quickly move to the sidewalk once th...,They are likely to remain on the street to con...,They are waiting for a gap in traffic to cross...,C,They are likely to remain on the street to con...
325,326,82,VRU22,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"The agent is intending to go to the left, but ...",They are waiting for a gap in traffic to cross...,They will leave the road as soon as the agent ...,They will quickly move to the sidewalk once th...,They are likely to remain on the street to con...,D,They are likely to remain on the street to con...
326,327,82,VRU22,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,"The agent is intending to go to the left, but ...",They are likely to remain on the street to con...,They are waiting for a gap in traffic to cross...,They will leave the road as soon as the agent ...,They will quickly move to the sidewalk once th...,A,They are likely to remain on the street to con...


In [92]:
safety_df

,index,g_index,id,image,question,A,B,C,D,answer,explanation
0,1,1,LRE01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,What is the purpose of the red-colored pavemen...,"It designates a special-use lane, the TransJak...",It marks an emergency lane that must be kept ...,It serves as a warning to drivers that the roa...,It indicates a high-speed or express lane for ...,A,"It designates a special-use lane, the TransJak..."
1,2,1,LRE01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,What is the purpose of the red-colored pavemen...,It indicates a high-speed or express lane for ...,"It designates a special-use lane, the TransJak...",It marks an emergency lane that must be kept ...,It serves as a warning to drivers that the roa...,B,"It designates a special-use lane, the TransJak..."
2,3,1,LRE01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,What is the purpose of the red-colored pavemen...,It serves as a warning to drivers that the roa...,It indicates a high-speed or express lane for ...,"It designates a special-use lane, the TransJak...",It marks an emergency lane that must be kept ...,C,"It designates a special-use lane, the TransJak..."
3,4,1,LRE01,iVBORw0KGgoAAAANSUhEUgAAC0AAAAcICAYAAACv5EAVAA...,What is the purpose of the red-colored pavemen...,It marks an emergency lane that must be kept ...,It serves as a warning to drivers that the roa...,It indicates a high-speed or express lane for ...,"It designates a special-use lane, the TransJak...",D,"It designates a special-use lane, the TransJak..."
4,5,2,LRE02,iVBORw0KGgoAAAANSUhEUgAABx0AAANvCAYAAAAIqsTbAA...,What is the specific purpose signified by the ...,Dedicated for bicycle.,The green color is used for aesthetic purposes...,"It marks a priority lane for public transport,...",To designate a legal stopping and passenger dr...,A,Dedicated for bicycle.
...,...,...,...,...,...,...,...,...,...,...,...
255,256,64,UTR09,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,What is the most appropriate driving action to...,Pull over to the left side and wait in case a ...,"Continue driving forward slowly, positioning t...",Honk and flash your headlights to alert any on...,Reverse and find an alternate route to avoid a...,B,"Continue driving forward slowly, positioning t..."
256,257,65,UTR10,iVBORw0KGgoAAAANSUhEUgAABuQAAANvCAYAAAAlfbxIAA...,As you prepare to enter the main road shown in...,Turn left to join the flow.,Turn right and drive against the flow of traffic.,Proceed with a U-turn in the current situation...,Drive straight ahead through the traffic flow.,A,Turn left to join the flow.
257,258,65,UTR10,iVBORw0KGgoAAAANSUhEUgAABuQAAANvCAYAAAAlfbxIAA...,As you prepare to enter the main road shown in...,Drive straight ahead through the traffic flow.,Turn left to join the flow.,Turn right and drive against the flow of traffic.,Proceed with a U-turn in the current situation...,B,Turn left to join the flow.
258,259,65,UTR10,iVBORw0KGgoAAAANSUhEUgAABuQAAANvCAYAAAAlfbxIAA...,As you prepare to enter the main road shown in...,Proceed with a U-turn in the current situation...,Drive straight ahead through the traffic flow.,Turn left to join the flow.,Turn right and drive against the flow of traffic.,C,Turn left to join the flow.
